In [250]:
#data
import yfinance as yf
import datetime
import numpy as np

def get_historical_prices(tickers, start_date, end_date):
    data = yf.download(tickers, start=start_date, end=end_date, progress=False)
    return data['Close']

# Define the tickers for the S&P 500 index and the 20 stocks
tickers = ['^GSPC', 'AAPL', 'MSFT', 'GOOGL', 'AMZN', 'FB', 'JPM', 'V', 'JNJ', 'UNH', 'BAC']#, 'TSLA', 'NVDA', 'WMT', 'PG', 'VZ', 'MA', 'HD', 'DIS', 'CMCSA']

# Calculate the start and end dates for the last six months
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=30)

# Retrieve the historical prices for the tickers
prices = get_historical_prices(tickers, start_date, end_date)

# Print the retrieved prices
print(prices)



1 Failed download:
['FB']: Exception('%ticker%: No timezone found, symbol may be delisted')


                           AAPL        AMZN        BAC  FB       GOOGL   
Date                                                                     
2023-10-16 00:00:00  178.720001  132.550003  26.990000 NaN  139.100006  \
2023-10-17 00:00:00  177.149994  131.470001  27.620001 NaN  139.720001   
2023-10-18 00:00:00  175.839996  128.130005  27.309999 NaN  137.960007   
2023-10-19 00:00:00  175.460007  128.399994  26.959999 NaN  137.750000   
2023-10-20 00:00:00  172.880005  125.169998  26.309999 NaN  135.600006   
2023-10-23 00:00:00  173.000000  126.559998  25.570000 NaN  136.500000   
2023-10-24 00:00:00  173.440002  128.559998  25.469999 NaN  138.809998   
2023-10-25 00:00:00  171.100006  121.389999  25.549999 NaN  125.610001   
2023-10-26 00:00:00  166.889999  119.570000  26.120001 NaN  122.279999   
2023-10-27 00:00:00  168.220001  127.739998  25.170000 NaN  122.169998   
2023-10-30 00:00:00  170.289993  132.710007  25.690001 NaN  124.459999   
2023-10-31 00:00:00  170.770004  133.0

In [257]:
import pyomo.environ as pyo

# create a model
model = pyo.ConcreteModel()


#model parameters
T = prices.shape[0]-1 #time duration
N = prices.shape[1]-1 #universe of assets #all dataset - index itself
V = np.array(prices.drop(['^GSPC'] ,axis = 1)).T
print(f'we have {N} assets and {T+1} periods')
# V = np.array(prices).T #value of asset i in period t

R = np.array([np.log(np.array(prices['^GSPC'][t])/np.array
(prices['^GSPC'][t-1])) if t >0 else 0 for t in range(T) ]) #return of index in period t
I = np.array(prices['^GSPC']) #value of index in period t

epsilon = 0.01
delta = 0.99
K = 5 #cardinality

gamma = 0.1 #portfolio of C that can be consumed for transaction costs
trans = 0.01 #amount of transaction if consumed to move from V[i,T]X[i] to V[i,T]x[i] or vice verca
C_cash = 1000 #$
model.X = np.random.randint(10,size = (N,)) #current tracking portfolio
print(f'our random initial portfolio: {model.X}')
#indices
model.i = np.arange(N)#index for stocks
model.t = np.arange(T)#index for time periods


# declare decision variables
# model.X = pyo.Var(model.i,domain=pyo.PositiveIntegers,bounds = (0,200)) #unit of asset i in current tracking portfolio
model.x = pyo.Var(model.i,domain=pyo.PositiveIntegers,bounds = (0,200)) #unit of asset i in new tracking portfolio
model.z = pyo.Var(model.i,domain = pyo.Binary) #1 if asset selected

#auxilary parameters and variables
M = np.inf
model.y = pyo.Var(model.t,domain = pyo.Binary) #1 for linearize maximum in objective function



#parameters with expressions
def r_expression_rule(model,t):
  return pyo.log(sum(V[i,t] * model.x[i] for i in range(N))/
                sum(V[i,t-1] * model.x[i] for i in range(N)))


model.r = pyo.Expression(model.t[1:], rule=r_expression_rule)




#C_cash,C_trans


#objective
def Obj(model):
  return 1/T * (sum(abs(model.r[t]-R[t])  for t in range(1,T)))
  # return 1/T * (sum((0.03-R[t])*model.y[t]  for t in range(1,T)))
  
  # return 1/T * (sum((model.r[t]-R[t])*model.y[t]  for t in range(1,T)))
  
  
  
#constraints
def Co1(model): #cardinality #3
  return sum(model.z[i] for i in range(N)) == K 
def Co2(model,i): #cardinality
  return ((V[i,T] * model.x[i]) / (sum(V[i,T]*model.X[i]  for i in range(N)) + C_cash)  ) <= model.z[i] * delta #4
  # return ((V[i,T] * model.x[i]) / C_cash  ) <= model.z[i] * delta #4
def Co3(model,i): #cardinality
  return ((V[i,T] * model.x[i]) / (sum(V[i,T]*model.X[i]  for i in range(N)) + C_cash)  ) >= model.z[i] * epsilon #4
  # return ((V[i,T] * model.x[i]) / C_cash ) >= model.z[i] * epsilon #4
def Co4(model,t): #linreaze the obj
  return (R[t] - model.r[t])  >= (5000000 * model.y[t]) - (5000000 * (1-model.y[t]) )
def Co5(model): #5 maximum amount that is allowed to use for transaction
  return sum(abs(model.x[i] - model.X[i])  * trans * V[i,T] for i in range(N)) <= gamma * (sum(V[i,T]*model.X[i]  for i in range(N))+C_cash) #5
  # return sum(x[i] * trans for i in range(N)) <= gamma * C_cash   #5
def Co6(model): #6
  return sum(V[i,T] * model.x[i] for i in range(N)) == (sum(V[i,T]*model.X[i]  for i in range(N)) + C_cash) - sum(abs(model.x[i]- model.X[i])  * trans * V[i,T] for i in range(N))
  
  
  
  
  
  

# def Co3(model):
#   return 
#constructing model
model.obj = pyo.Objective(rule = Obj,sense = pyo.minimize)
model.Co1 = pyo.Constraint(rule = Co1)
model.Co2 = pyo.Constraint(model.i,rule = Co2)
model.Co3 = pyo.Constraint(model.i,rule = Co3)
model.Co4 = pyo.Constraint(model.t[1:],rule = Co4)
# model.Co5 = pyo.Constraint(rule = Co5)
# model.Co6 = pyo.Constraint(rule = Co6)


DEBUG: Constructing ConcreteModel 'ConcreteModel', from data=None
we have 10 assets and 21 periods
our random initial portfolio: [5 2 1 9 4 3 1 2 9 9]
DEBUG: Constructing Set, name=FiniteScalarSet, from data=None
DEBUG: Constructing IndexedVar 'x' on [Model] from data=None
DEBUG: Constructing Variable x
DEBUG: Constructed component ''[Model].x'':
    x : Size=10, Index=x_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     1 :  None :   200 : False :  True : PositiveIntegers
          1 :     1 :  None :   200 : False :  True : PositiveIntegers
          2 :     1 :  None :   200 : False :  True : PositiveIntegers
          3 :     1 :  None :   200 : False :  True : PositiveIntegers
          4 :     1 :  None :   200 : False :  True : PositiveIntegers
          5 :     1 :  None :   200 : False :  True : PositiveIntegers
          6 :     1 :  None :   200 : False :  True : PositiveIntegers
          7 :     1 :  None :   200 : False :  True : Positive

DEBUG: Constructing Expression, name=r, from data=None
DEBUG: Constructed component ''[Model].r'':
    r : Size=19, Index=r_index
        Key : Expression
          1 : log((177.14999389648438*x[0] + 131.47000122070312*x[1] +
              27.6200008392334*x[2] + nan*x[3] + 139.72000122070312*x[4] +
              156.08999633789062*x[5] + 147.52999877929688*x[6] +
              332.05999755859375*x[7] + 536.6500244140625*x[8] +
              241.1999969482422*x[9])/(178.72000122070312*x[0] +
              132.5500030517578*x[1] + 26.989999771118164*x[2] + nan*x[3] +
              139.10000610351562*x[4] + 157.52999877929688*x[5] +
              147.85000610351562*x[6] + 332.6400146484375*x[7] +
              538.030029296875*x[8] + 240.07000732421875*x[9]))
          2 : log((175.83999633789062*x[0] + 128.1300048828125*x[1] +
              27.309999465942383*x[2] + nan*x[3] + 137.9600067138672*x[4] +
              152.72999572753906*x[5] + 145.91000366210938*x[6] +
              330.10

In [258]:
# import logging
# logging.getLogger('pyomo.core').setLevel(logging.DEBUG)
#solvers
# solver = pyo.SolverFactory('glpk',executable = 'C:\glpk-4.65\w64\glpsol')
solver = pyo.SolverFactory('gams')
results = solver.solve(model)
# pyo.SolverFactory('mindtpy').solve(model,
#                                    strategy='GOA',
#                                    mip_solver='cplex',
#                                    nlp_solver='scip')
model.display()
# results

DEBUG: Writing model 'unknown' to file
'C:\Users\danie\AppData\Local\Temp\tmp78wfo6x2\model.gms' with format gams
ERROR: GAMS encountered an error during solve. Check listing file for details.
ERROR:
ERROR: GAMS Listing file:


    GAMS 24.1.2  r40979 Released Jun 16, 2013 WEX-WEI x86_64/MS Windows
    11/14/23 23:06:56 Page 1 G e n e r a l   A l g e b r a i c   M o d e l i n
    g   S y s t e m C o m p i l a t i o n


      96  c2_hi.. nan*x11 + (-0.98999999999999999)*x1 =l= 0 ;
    ****            $140
     163  SOLVE GAMS_MODEL USING minlp minimizing GAMS_OBJECTIVE;
    ****                                                        $257
     194  put x1 ' ' x1.l ' ' x1.m /;
    ****                $141
     195  put x2 ' ' x2.l ' ' x2.m /;
    ****                $141
     196  put x3 ' ' x3.l ' ' x3.m /;
    ****                $141
     197  put x4 ' ' x4.l ' ' x4.m /;
    ****                $141
     198  put x5 ' ' x5.l ' ' x5.m /;
    ****                $141
     199  put x6 ' '

RuntimeError: GAMS encountered an error during solve. Check listing file for details.